In [1]:
import json
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN, GRU, LSTM
from keras.layers.core import Dense, Dropout
from keras.layers.wrappers import TimeDistributed
from keras.layers import Convolution1D, MaxPooling1D
from keras.utils import np_utils
import itertools
import ast
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import sys, os
import traceback
from itertools import chain
import numpy as np
import pandas as pd

import tensorflow as tf

import matplotlib.pyplot as plt

%matplotlib inline

Using TensorFlow backend.


In [2]:
def get_sentences(data_list):
    sentences = []
    for a in data_list.article.unique():
        words = []
        prev_n = 0
        for _,s in data_list.loc[data_list.article == a].iterrows():
            n = s['sentence_no']
            if n != prev_n:
                if len(words) > 2 and len(set([x[1] for x in words])) > 1:
                    sentences.append(words)
                words = []
            words.append((s['word'], s['encoding'], s['part_of_speech']))
            prev_n = n
            
        if len(words) > 2 and len(set([x[1] for x in words])) > 1:
            sentences.append(words)
            
    return sentences

In [41]:
data_list = pd.read_csv('../data/go/go_cc3.csv')
data_list.word.fillna('NA', inplace=True)

temp = pd.read_csv('../data/go/go_bp3.csv')
temp.word.fillna('NA', inplace=True)

temp1 = pd.read_csv('../data/go/go_mf3.csv')
temp1.word.fillna('NA', inplace=True)
temp1 = temp1.rename(columns={'encoding' : 'encoding_z'})

data_list = data_list.merge(temp, on=['article', 'sentence_no', 'word_no'])
data_list = data_list.merge(temp1, on=['article', 'sentence_no', 'word_no'])
data_list.sort_values(['article', 'sentence_no'], kind='mergesort', inplace=True)
x_counts = data_list.encoding_x.value_counts()
y_counts = data_list.encoding_y.value_counts()
z_counts = data_list.encoding_z.value_counts()
# data_list['encoding_x'] = data_list.encoding_x.apply(lambda x: x if x_counts[x] > 4 else 'O')
# data_list['encoding_y'] = data_list.encoding_y.apply(lambda x: x if y_counts[x] > 4 else 'O')
# data_list['encoding'] = [i if (i == 'O' and j == 'O') else j for i,j in zip(data_list.encoding_x, data_list.encoding_y)]
temp_encoding = []
multi_ann_sents = []
for i,j,k,art,sent in zip(data_list.encoding_x, data_list.encoding_y, data_list.encoding_z, data_list.article, data_list.sentence_no):
    if i != 'O' and j != 'O':
        multi_ann_sents.append((art, sent))
    if i == j and j == k:
        temp_encoding.append('O')
    elif i == 'O' and j != 'O':
        temp_encoding.append(j)
    elif j == 'O' and i != 'O':
        temp_encoding.append(i)
    else:
        temp_encoding.append(k)
# data_list['encoding'] = temp_encoding
# val_cnt = data_list.encoding.value_counts()
# data_list['encoding'] = data_list.encoding.apply(lambda x: 'O' if val_cnt[x] < 5 else x)
# data_list = data_list.rename(columns={'word_x' : 'word'})

In [46]:
(pd.Series(multi_ann_sents).value_counts() > 1)

(17069463.txt, 27)      True
(16628246.txt, 133)     True
(17083276.txt, 27)      True
(17083276.txt, 245)     True
(16221973.txt, 293)     True
(16221973.txt, 273)     True
(17696610.txt, 268)     True
(15550985.txt, 215)     True
(16221973.txt, 253)     True
(16098226.txt, 139)     True
(16221973.txt, 169)     True
(15207008.txt, 206)     True
(15836427.txt, 326)     True
(15207008.txt, 158)     True
(17083276.txt, 4)       True
(11597317.txt, 69)      True
(15917436.txt, 234)     True
(15588329.txt, 192)     True
(16628246.txt, 8)       True
(15588329.txt, 11)      True
(16109169.txt, 21)      True
(12925238.txt, 201)     True
(16221973.txt, 263)     True
(17069463.txt, 30)      True
(16221973.txt, 195)     True
(17069463.txt, 258)     True
(15207008.txt, 248)     True
(15207008.txt, 232)     True
(14737183.txt, 26)      True
(16221973.txt, 283)     True
                       ...  
(16628246.txt, 214)    False
(17447844.txt, 210)    False
(17244351.txt, 172)    False
(15819996.txt,

In [48]:
data_list[(data_list.article == '17069463.txt') & (data_list.sentence_no == 27)][['article','sentence_no', 'word_x', 'encoding_x', 'encoding_y']]

,article,sentence_no,word_x,encoding_x,encoding_y
419903,17069463.txt,27,Several,O,O
419904,17069463.txt,27,roles,O,O
419905,17069463.txt,27,of,O,O
419906,17069463.txt,27,RanBP2,O,O
419907,17069463.txt,27,have,O,O
419908,17069463.txt,27,emerged,O,O
419909,17069463.txt,27,that,O,O
419910,17069463.txt,27,implicate,O,O
419911,17069463.txt,27,RanBP2,O,O
419912,17069463.txt,27,in,O,O
